In [3]:
"""
Detects and analyzes the binding events that would result in NMR-STD signals (not water mediated)
"""

'\nDetects and analyzes the binding events that would result in NMR-STD signals (not water mediated)\n'

In [4]:
XTC = "NP22don-53/NP22don-53_PRO1-11_FIX100ns.xtc"
TPR = "NP22don-53/NP22don-53_PRO1.tpr"
NAME = XTC[:-8]

In [5]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from MDAnalysis import *
from collections import defaultdict

In [6]:
U = Universe(TPR, XTC)
print(len(U.trajectory))
DT = U.trajectory[0].dt
print(DT)

5001
20.0


In [123]:
identical_mono_H = ["H4 H5", "H6 H7", "H8 H9", "H10 H11", "H12 H13", "H14 H15", "H16 H17", "H20 H21", "H19 H22"]
identical_DON_H = ["H1", "H2 H3", "H6 H7", "H8 H9"]

sel = {
"all_gold"  : U.select_atoms("name AU AUS AUL"),
"mono_H"    : [U.select_atoms("resname L22 and name {}".format(ident)) for ident in identical_mono_H],
"DON_H"     : [U.select_atoms("resname DON and name {}".format(ident)) for ident in identical_DON_H],
"SOL"       : U.select_atoms("resname SOL")
}

In [126]:
props_bind_time = {
'anchor'    : 'all_gold', #the script reports the median distance respect to this group
'ref'       : "mono_H",
'targets'    : ["DON_H"],
'solvent'    : "SOL",
'start_ps'  : 0,
'stop_ps'   : 1000,
'd_max'     : 4, #A, threshold distance for magnetization transfer
}

In [127]:
class STDFrame:
    def __init__(self, time, bound_pairs, dists):
        self.time = time
        self.pairs = bound_pairs
        self.dists = dists

class STDEvent:
    def __init__(self, pair, dist, ndx_frame, counter, res):
        self.res = res
        self.pair = pair
        self.counter = counter
        self.dist = [dist]
        self.ndx_fr = [ndx_frame]
        
    def extend_event(self, dist, ndx_frame):
        self.dist.append(dist)
        self.ndx_fr.append(ndx_frame)
        
    def compress_event(self, func=np.median):
        self.anchor_dist = func(self.dist)
        self.duration = DT*(max(self.ndx_fr) - min(self.ndx_fr) + 1)

class STDTrajectory:
    def __init__(self, frames):
        self.frames = frames
    
    def Search_STDEvents(self):
        self.events = {}
        counters = defaultdict(int)
        for f, frame in enumerate(self.frames):
            print(frame.time, end="\r")
            for res, dist in frame.dists.items():
                for pair in frame.pairs[res]:
                    key = (*pair, counters[pair])
                    if key not in self.events.keys():
                        self.events[key] = STDEvent(pair, dist, f, counters[pair], res)
                    else:
                        if f-1 in self.events[key].ndx_fr:
                            self.events[key].extend_event(dist, f)
                        else:
                            counters[pair] += 1
                            self.events[key] = STDEvent(pair, dist, f, counters[pair], res)
                            
        for event in self.events.values():
            event.compress_event()

In [119]:
def binding_time_std(props):
    all_trajectories = {}
    g_anchor = sel[props['anchor']]
    for target in props['targets']:
        g_target = sel[target]
        g_ref = sel[props['ref']]
        n_targets = len(g_target.residues)
        frames = []
        for ts in U.trajectory:
            if ts.time >= props['stop_ps']:
                break
            elif ts.time >= props['start_ps']:
                print(ts.time, end="\r")            
                pairs = defaultdict(list)

                anchor_dist = {res.ix : np.linalg.norm(res.atoms.center_of_mass()-g_anchor.center_of_mass()) for res in g_target.residues}
                dists = cdist(g_ref.positions, g_target.positions)

                ndx_close = np.where(dists<=props['d_max'])
                ndx_ref_H = g_ref[ndx_close[0]].ix
                target_H = g_target[ndx_close[1]]
                ndx_target_H = target_H.ix
                res_target = [U.atoms[ndx].resid for ndx in ndx_target_H]

                for rt, nth, nrh in zip(res_target, ndx_target_H, ndx_ref_H):
                    pairs[rt].append((nrh, nth))

                stdframe = STDFrame(ts.time, pairs, anchor_dist)
                frames.append(stdframe)

            all_trajectories[target] = STDTrajectory(frames)
    return all_trajectories   

trajs = binding_time_std(props_bind_time)

In [120]:
trajs['DON_H'].Search_STDEvents()

In [130]:
print([ev.duration for ev in trajs['DON_H'].events.values() if ev.duration>500])

[520.0, 720.0, 560.0, 560.0, 680.0, 560.0, 520.0, 600.0, 560.0, 620.0, 540.0, 540.0, 820.0, 680.0]
